# 📰 어린이 뉴스 데이터 요약패턴 개발

## 1. data import

In [1]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df = pd.read_csv('./kid_news.csv', index_col=[0])

In [3]:
# 어린이 뉴스 파일을 load
df.head()

,news_title,news_subtitle,news_article,news_url
0,"[세상 속으로] ""러시아 고양이는 안 됩니다""","국제고양이연맹, '월드 캣 쇼'서 퇴출",전 세계가 우크라이나를 침공(侵攻)한 러시아에 각종 제재를 가하고 있다. 이에 러시...,http://kid.chosun.com/site/data/html_dir/2022/...
1,[뉴스탐험대] 홀로 1000㎞… 우크라 國境<국경> 탈출한 11세 소년,NaN,우크라이나 소년이 홀로 국경(國境)을 넘은 뭉클한 연이 전해졌다. 영국 매체 가디언...,http://kid.chosun.com/site/data/html_dir/2022/...
2,[뉴스탐험대] 온라인 쇼핑몰 '니켈·납투성이 장신구' 주의,NaN,온라인 쇼핑몰에서 '무(無)알러지' '무니켈 도금' 등으로 홍보·판매되는 액세서리에...,http://kid.chosun.com/site/data/html_dir/2022/...
3,"[뉴스탐험대] ""코로나에 강한 혈액형 있다""… O형, 위험 최저",NaN,혈액형이 O형(型)인 람이 코로나 바이러스 감염 시 망·위중증 위험이 다른 혈액형보...,http://kid.chosun.com/site/data/html_dir/2022/...
4,잡히지 않는 火魔<화마>… 문화재 등 대규모 피해,서울 면적 4분의 1 이상 불타,최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림(山林) 피해가 우려되고 있...,http://kid.chosun.com/site/data/html_dir/2022/...


In [4]:
df.iloc[:, :3]

,news_title,news_subtitle,news_article
0,"[세상 속으로] ""러시아 고양이는 안 됩니다""","국제고양이연맹, '월드 캣 쇼'서 퇴출",전 세계가 우크라이나를 침공(侵攻)한 러시아에 각종 제재를 가하고 있다. 이에 러시...
1,[뉴스탐험대] 홀로 1000㎞… 우크라 國境<국경> 탈출한 11세 소년,NaN,우크라이나 소년이 홀로 국경(國境)을 넘은 뭉클한 연이 전해졌다. 영국 매체 가디언...
2,[뉴스탐험대] 온라인 쇼핑몰 '니켈·납투성이 장신구' 주의,NaN,온라인 쇼핑몰에서 '무(無)알러지' '무니켈 도금' 등으로 홍보·판매되는 액세서리에...
3,"[뉴스탐험대] ""코로나에 강한 혈액형 있다""… O형, 위험 최저",NaN,혈액형이 O형(型)인 람이 코로나 바이러스 감염 시 망·위중증 위험이 다른 혈액형보...
4,잡히지 않는 火魔<화마>… 문화재 등 대규모 피해,서울 면적 4분의 1 이상 불타,최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림(山林) 피해가 우려되고 있...
...,...,...,...
1670,돼지 뇌에 컴퓨터 칩 연결… 생각만으로 기계 움직일 수 있을까?,NaN,"머스크, 유튜브에 실험 돼지 '거트루드' 공개 ""뇌 질환 치료 도움… 올해 안에 람..."
1671,탑승형 로봇 '프로스테시스' 조종사를 뽑습니다!,NaN,"람의 팔다리에 연결해 조종 ""로봇 활용한 경주 스포츠 만들 것"" 람이 탑승한 로봇끼..."
1672,"모험 떠날 준비 됐니? 그럼, 책장을 펼쳐봐!",NaN,띵똥! 선물이 도착했습니다. 판타지 소설 '해리포터'의 작가 조앤 롤링(55·영국)...
1673,"방탄소년단·블랙핑크, 세계 음원시장 '올킬'",NaN,아이돌 그룹 방탄소년단(BTS)과 블랙핑크가 세계 무대를 뒤흔들고 있다. 글로벌 시...


In [5]:
# 데이터 형태 확인
print(list(df.columns))
print(df.shape)

['news_title', 'news_subtitle', 'news_article', 'news_url']
(1675, 4)


---

## 2. 데이터 전처리(Data Preprocessing)

### 결측치 확인

In [6]:
# 데이터 정보를 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1675 entries, 0 to 1674
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_title     1675 non-null   object
 1   news_subtitle  403 non-null    object
 2   news_article   1636 non-null   object
 3   news_url       1675 non-null   object
dtypes: object(4)
memory usage: 65.4+ KB


In [7]:
# 결측값 확인
df.isnull().sum()

news_title          0
news_subtitle    1272
news_article       39
news_url            0
dtype: int64

In [8]:
# null/NaN to ""
df = df.fillna("")
# df = df.dropna(subset=['news_article','news_subtitle'])

In [9]:
# 결측값 확인
df.isna().sum()

news_title       0
news_subtitle    0
news_article     0
news_url         0
dtype: int64

---

### 문자열 길이 확인 및 추가

In [10]:
# df["news_len"] = df.news_article.str.len()
df['news_len'] = df['news_article'].apply(lambda x: len(x))

In [11]:
df.head()

,news_title,news_subtitle,news_article,news_url,news_len
0,"[세상 속으로] ""러시아 고양이는 안 됩니다""","국제고양이연맹, '월드 캣 쇼'서 퇴출",전 세계가 우크라이나를 침공(侵攻)한 러시아에 각종 제재를 가하고 있다. 이에 러시...,http://kid.chosun.com/site/data/html_dir/2022/...,469
1,[뉴스탐험대] 홀로 1000㎞… 우크라 國境<국경> 탈출한 11세 소년,,우크라이나 소년이 홀로 국경(國境)을 넘은 뭉클한 연이 전해졌다. 영국 매체 가디언...,http://kid.chosun.com/site/data/html_dir/2022/...,380
2,[뉴스탐험대] 온라인 쇼핑몰 '니켈·납투성이 장신구' 주의,,온라인 쇼핑몰에서 '무(無)알러지' '무니켈 도금' 등으로 홍보·판매되는 액세서리에...,http://kid.chosun.com/site/data/html_dir/2022/...,446
3,"[뉴스탐험대] ""코로나에 강한 혈액형 있다""… O형, 위험 최저",,혈액형이 O형(型)인 람이 코로나 바이러스 감염 시 망·위중증 위험이 다른 혈액형보...,http://kid.chosun.com/site/data/html_dir/2022/...,451
4,잡히지 않는 火魔<화마>… 문화재 등 대규모 피해,서울 면적 4분의 1 이상 불타,최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림(山林) 피해가 우려되고 있...,http://kid.chosun.com/site/data/html_dir/2022/...,620


In [12]:
# min 값을 봤을 때 기사가 없는 경우도 존재
df.describe().T

,count,mean,std,min,25%,50%,75%,max
news_len,1675.0,981.068657,724.715977,0.0,479.0,772.0,1221.0,9658.0


---

### 이상치 확인
- 이상한 뉴스 데이터 확인

In [13]:
# 기사 길이 200 이하 확인
condition = (df.news_len <= 200)

shorts_df = df[condition][['news_title','news_url','news_len']].sort_values(by='news_len' ,ascending=True)
shorts_df

,news_title,news_url,news_len
8,[메시지 뉴스] 명태·대게와 러시아,http://kid.chosun.com/site/data/html_dir/2022/...,0
303,[메시지 뉴스] 동지(冬至),http://kid.chosun.com/site/data/html_dir/2021/...,0
325,[메시지 뉴스] 성탄 트리의 기원,http://kid.chosun.com/site/data/html_dir/2021/...,0
389,[메시지 뉴스] 일본 한류 열풍,http://kid.chosun.com/site/data/html_dir/2021/...,0
400,[메시지 뉴스] 우주탐사 인형,http://kid.chosun.com/site/data/html_dir/2021/...,0
423,[메시지 뉴스] 대학수학 능력시험,http://kid.chosun.com/site/data/html_dir/2021/...,0
451,[메시지 뉴스] COP26,http://kid.chosun.com/site/data/html_dir/2021/...,0
485,[메시지 뉴스] 기초 학력과 학교생활,http://kid.chosun.com/site/data/html_dir/2021/...,0
504,[메시지 뉴스] 대통령 선거,http://kid.chosun.com/site/data/html_dir/2021/...,0
536,[메시지 뉴스] 탄소중립,http://kid.chosun.com/site/data/html_dir/2021/...,0


In [14]:
shorts_df.shape

(60, 3)

In [15]:
short_title = shorts_df.news_title
list(short_title)

['[메시지 뉴스] 명태·대게와 러시아',
 '[메시지 뉴스] 동지(冬至)',
 '[메시지 뉴스] 성탄 트리의 기원',
 '[메시지 뉴스] 일본 한류 열풍',
 '[메시지 뉴스] 우주탐사 인형',
 '[메시지 뉴스] 대학수학 능력시험',
 '[메시지 뉴스] COP26',
 '[메시지 뉴스] 기초 학력과 학교생활',
 '[메시지 뉴스] 대통령 선거',
 '[메시지 뉴스] 탄소중립',
 '[메시지 뉴스] 골프',
 '[메시지 뉴스] 국정감사',
 '[메시지 뉴스] 기네스북',
 '[메시지 뉴스] 추석',
 '[메시지 뉴스] 아프간 전쟁',
 '[메시지 뉴스] 코로나 백신',
 '[요리 쿡! 조리 쿡!] 시원하고 달콤하고 코코아 아이스바',
 '[요리 쿡! 조리 쿡!] 호호 불어 드세요 돌돌 말린 채소 춘권',
 '[요리 쿡! 조리 쿡!] 콕콕 찍어 입으로 쏙쏙 톡톡 튀는 팝콘 치킨',
 "[요리 쿡! 조리 쿡!] 파티 음식으로도 'OK' 토마토 브루스케타",
 '[메시지 뉴스] 세계의 새해 풍습',
 '[메시지 뉴스] 성년(成年) 기준',
 '기자들의 수다',
 '[영상-맛있는 실험실] 감자채 피자를 만들어 봤어요‼',
 '[메시지 뉴스] 체스',
 '[메시지 뉴스] 용감하게 질문하기!',
 '[메시지 뉴스] 어린이 교통안전',
 "[꿀beetv] 안철수 후보의 '어린이를 위한 대선공약'은?",
 '[꿀beetv] 어린이조선일보 기자들의 해보GO! "별걸 다 꾸며요"',
 '[메시지 뉴스] 한국판 미네르바 大學',
 '[꿀BEE] 강아지 건강에 최고인 수제 간식 만들기',
 '[신자영의 5분 영어] 모니 모니 석가모니 일화 속 영어 표현 배우기!',
 '[일상 속 간단한 운동] 본지 기자들, 스트레칭에 도전하다',
 '[메시지 뉴스] 올바른 양치법',
 '[메시지 뉴스] 어린이 안전 관리 강화',
 '[영상-출동! 명예기자] 국립과학수사연구원을 체험하다!',
 "[영상-출동! 명예기자] 화제의 인물 글 쓰는 의사 '남궁인'을 만나다!",
 '[메시지 뉴스] 재밌는

---

### 데이터 삭제
- 시리즈 기사 title과 그 외 text title 확인

In [16]:
# 전체 기사 대상 진행
split_title = []

for title in df.news_title:
    # m = re.search(r'(?<=\[).*(?=\])', title)
    m = re.search(r'\[(.*?)\]', title)
    if m :
        split_title.append(m.group())
    else:
        # sub_text = re.sub(r'[?]',"\?",title)
        split_title.append(title)

In [17]:
df['split_title'] = split_title

In [18]:
df[['split_title']][:10]

,split_title
0,[세상 속으로]
1,[뉴스탐험대]
2,[뉴스탐험대]
3,[뉴스탐험대]
4,잡히지 않는 火魔<화마>… 문화재 등 대규모 피해
5,[뉴스탐험대]
6,"[우리의 보금자리, 놀라운 住居 상식]"
7,[주목! 정보 레터]
8,[메시지 뉴스]
9,[뉴스탐험대]


In [19]:
# 시리즈 기사 유니크 값 체크
count_unique_title = df.split_title.value_counts()
count_unique_title[count_unique_title >= 2]

[뉴스탐험대]                  544
[오늘의 키워드]                 86
기자들의 수다                   77
[메시지 뉴스]                  26
[팝콘뉴스]                    26
[주목! 정보 레터]               19
[푸른 별, 지구를 지켜라]           15
[ESG 기업 탐방]               14
[이슬기의 슬기로운 스토리 잡스]        10
[의학 정보 쏙쏙, 우리 아이 쑥쑥]       9
[여러분의 생각은?]                8
[기획]                       8
[사진으로 보는 세상]               7
[알차고 통통튀는 인터뷰]             7
[현장]                       5
[호국 보훈의 달 특집]              5
[기자 수첩]                    5
[출동! 명예기자]                 4
[탄소 중립 기획]                 4
[요리 쿡! 조리 쿡!]              4
[세상 속으로]                   4
[초등 트렌드]                   4
[꿀BeeTV]                   4
[알립니다]                     3
[영상-출동! 명예기자]              3
[현장 스케치]                   3
[우리의 보금자리, 놀라운 住居 상식]      3
[초등 줌터뷰]                   3
[꿀beetv]                   3
[생생~~영상뉴스]                 2
[현장 취재]                    2
간추린 소식                     2
[뉴스타그램]                    2
Name: split_title, dtype: int64

In [20]:
# 200줄 이하 기사 대상 진행
import re

split_title = []

for title in short_title:
    m = re.search(r'\[(.*?)\]', title)
    if m :
        split_title.append(m.group())
    else:
        split_title.append(title)

In [21]:
list(set(split_title))

['[꿀beetv]',
 '기자들의 수다',
 '[일상 속 간단한 운동]',
 '잠자리 드론과 풍선 망원경? 과감한 박새의 사연은?',
 '[신자영의 5분 영어]',
 '[생생 인터뷰 현장!]',
 '[POPCORN NEWS]',
 '신자영의 5분 영어!',
 '[정답 공개]',
 '[생생~~영상뉴스]',
 '[꿀BEE]',
 '노릇하게 구워볼까? 폭신폭신 팬케이크',
 '초콜릿 잼 품은 바나나 크레이프',
 '[요리 쿡! 조리 쿡!]',
 "편하게 들고 '쏙쏙' 한입 토스트 꼬치",
 '기록, 또 기록… 케이팝에 한계란 없다',
 '[영상-출동! 명예기자]',
 '[카드뉴스]',
 '[동영상]',
 '[꿀BeeTV]',
 '[메시지 뉴스]',
 '[영상-맛있는 실험실]',
 '돌돌 말렸네! 맛있는 시금치 롤',
 '간편하게 한 끼 챙길 땐 쫀득쫀득 그래놀라 바']

In [22]:
# 전체 기사에 대한 불용어 처리를 진행합니다
print('불용어 처리 전 df 크기:',len(df))

불용어 처리 전 df 크기: 1675


In [23]:
for title in split_title:
    # 불용어 단어가 news_title에 포함되지 않은 데이터를 걸러냅니다
    df = df[df['news_title'].apply(lambda x: title not in x)]
    # 간혹 시리즈 타이틀이 기사 본문 안에서 시작하는 경우도 있습니다
    df = df[df['news_article'].apply(lambda x: title not in x)]

In [24]:
print('불용어 처리 후 df 크기:',len(df))

불용어 처리 후 df 크기: 1538


In [25]:
# 기사의 길이가 300개 아래인 경우 버림
# 문장 요약을 위해 세 문장보다 많은 기사를 확보하기 위함
df = df[df['news_len'] > 300].reset_index(drop=True)
df.head(2)

,news_title,news_subtitle,news_article,news_url,news_len,split_title
0,"[세상 속으로] ""러시아 고양이는 안 됩니다""","국제고양이연맹, '월드 캣 쇼'서 퇴출",전 세계가 우크라이나를 침공(侵攻)한 러시아에 각종 제재를 가하고 있다. 이에 러시...,http://kid.chosun.com/site/data/html_dir/2022/...,469,[세상 속으로]
1,[뉴스탐험대] 홀로 1000㎞… 우크라 國境<국경> 탈출한 11세 소년,,우크라이나 소년이 홀로 국경(國境)을 넘은 뭉클한 연이 전해졌다. 영국 매체 가디언...,http://kid.chosun.com/site/data/html_dir/2022/...,380,[뉴스탐험대]


In [26]:
# 초기 데이터 : 1675
print('짧은 기사 처리 후 df 크기:',len(df))

짧은 기사 처리 후 df 크기: 1527


---

### 특수문자 및 공백 제거

**0. Basic**
- 기초적인 전처리,
- html tag 제거(크롤링한 데이터일경우)
- 숫자, 영어, 특수문자 등 필요하지 않은 언어 제거
- Lowercasing
- "@%*=()/+ 와 같은 punctuation(문장부호) 제거
- 한자 - 삭제할지 고민


In [27]:
import re 
def clean_text(text): 
    text = re.sub(r'(\[)(.*?)(\])','',str(text)) # remove [title]
    text = re.sub(r'(\()(.*?)(\))', '', text)  # 소괄호 (세부 설명
    text = re.sub(r'[?!]', '.', text)          # ?! => 마침표 처리
    text = re.sub(r'[\,\·\:\-\_]', ' ', text)  # 문장부호 구분자 => 공백 처리
    # text = text.lower() #lower case 
    text = re.sub(r'\s+', ' ', text) #remove extra space
    # text = re.sub(r'<[^>]+>','',text) #remove Html tags 
    text = re.sub(r'<[^가-힣]+>','',text)# 홑화살괄호 => 한글만 빼고 제거
    text = re.sub(r'^\s+', '', text) #remove space from start
    text = re.sub(r'\s+$', '', text) #remove space from the end
    text = re.sub('[^가-힣\w. ]', '', text)    # 한글, 문자, 숫자, 마침표, 공백 제외 제거
    text = re.sub(r'[一-龥]', '', text) #remove Chinese character
    text = re.sub('\s{2,}', ' ', text)        # 2번 이상의 space 제거
    text = text.strip()
    return text

In [28]:
text = ' <p>[세상 속으로] <화마> "고양이?  러시아·고양이(소말리·스핑크스·오리엔탈 등...)는 안  됩니다.無 </p>" '
clean_text(text)

'화마 고양이. 러시아 고양이는 안 됩니다.'

In [29]:
df["news_title"] = df["news_title"].apply(clean_text)
df["news_subtitle"] = df["news_subtitle"].apply(clean_text)
df["news_article"] = df["news_article"].apply(clean_text)

In [30]:
df.iloc[:, ]

,news_title,news_subtitle,news_article,news_url,news_len,split_title
0,러시아 고양이는 안 됩니다,국제고양이연맹 월드 캣 쇼서 퇴출,전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다. 이에 러시아 고양...,http://kid.chosun.com/site/data/html_dir/2022/...,469,[세상 속으로]
1,홀로 1000 우크라 국경 탈출한 11세 소년,,우크라이나 소년이 홀로 국경을 넘은 뭉클한 연이 전해졌다. 영국 매체 가디언과 메트...,http://kid.chosun.com/site/data/html_dir/2022/...,380,[뉴스탐험대]
2,온라인 쇼핑몰 니켈 납투성이 장신구 주의,,온라인 쇼핑몰에서 무알러지 무니켈 도금 등으로 홍보 판매되는 액세서리에서 기준치를 ...,http://kid.chosun.com/site/data/html_dir/2022/...,446,[뉴스탐험대]
3,코로나에 강한 혈액형 있다 O형 위험 최저,,혈액형이 O형인 람이 코로나 바이러스 감염 시 망 위중증 위험이 다른 혈액형보다 낮...,http://kid.chosun.com/site/data/html_dir/2022/...,451,[뉴스탐험대]
4,잡히지 않는 화마 문화재 등 대규모 피해,서울 면적 4분의 1 이상 불타,최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림 피해가 우려되고 있다. 지...,http://kid.chosun.com/site/data/html_dir/2022/...,620,잡히지 않는 火魔<화마>… 문화재 등 대규모 피해
...,...,...,...,...,...,...
1522,돼지 뇌에 컴퓨터 칩 연결 생각만으로 기계 움직일 수 있을까.,,머스크 유튜브에 실험 돼지 거트루드 공개 뇌 질환 치료 도움 올해 안에 람 적용 손...,http://kid.chosun.com/site/data/html_dir/2020/...,1124,돼지 뇌에 컴퓨터 칩 연결… 생각만으로 기계 움직일 수 있을까?
1523,탑승형 로봇 프로스테시스 조종사를 뽑습니다.,,람의 팔다리에 연결해 조종 로봇 활용한 경주 스포츠 만들 것 람이 탑승한 로봇끼리 ...,http://kid.chosun.com/site/data/html_dir/2020/...,721,탑승형 로봇 '프로스테시스' 조종사를 뽑습니다!
1524,모험 떠날 준비 됐니. 그럼 책장을 펼쳐봐.,,띵똥. 선물이 도착했습니다. 판타지 소설 해리포터의 작가 조앤 롤링이 코로나19로 ...,http://kid.chosun.com/site/data/html_dir/2020/...,1668,"모험 떠날 준비 됐니? 그럼, 책장을 펼쳐봐!"
1525,방탄소년단 블랙핑크 세계 음원시장 올킬,,아이돌 그룹 방탄소년단과 블랙핑크가 세계 무대를 뒤흔들고 있다. 글로벌 시장을 겨냥...,http://kid.chosun.com/site/data/html_dir/2020/...,756,"방탄소년단·블랙핑크, 세계 음원시장 '올킬'"


---

## 3. news_article 문장 분리

### split() 사용

In [31]:
# (\. ) 온점으로 분리
df["split_article"] = df.news_article.str.split('\. ')

In [32]:
df["split_article"]

0       [전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다, 이에 러시아 고...
1       [우크라이나 소년이 홀로 국경을 넘은 뭉클한 연이 전해졌다, 영국 매체 가디언과 메...
2       [온라인 쇼핑몰에서 무알러지 무니켈 도금 등으로 홍보 판매되는 액세서리에서 기준치를...
3       [혈액형이 O형인 람이 코로나 바이러스 감염 시 망 위중증 위험이 다른 혈액형보다 ...
4       [최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림 피해가 우려되고 있다, ...
                              ...                        
1522    [머스크 유튜브에 실험 돼지 거트루드 공개 뇌 질환 치료 도움 올해 안에 람 적용 ...
1523    [람의 팔다리에 연결해 조종 로봇 활용한 경주 스포츠 만들 것 람이 탑승한 로봇끼리...
1524    [띵똥, 선물이 도착했습니다, 판타지 소설 해리포터의 작가 조앤 롤링이 코로나19로...
1525    [아이돌 그룹 방탄소년단과 블랙핑크가 세계 무대를 뒤흔들고 있다, 글로벌 시장을 겨...
1526    [인공 지능 의가 있는 병원을 상상한 적 있니, 놀랍게도 벌써 우리나라 한 병원에서...
Name: split_article, Length: 1527, dtype: object

In [33]:
df.news_article[0]

'전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다. 이에 러시아 고양이도 국제 대회에 출전하지 못한다는 소식이 전해졌다. 지난 3일 영국 일간 데일리메일에 따르면 70여 년 역를 가 국제고양이연맹이 최근 성명을 통해 월드 캣 쇼에 러시아 소속 고양이 참가를 금지하겠다고 밝혔다. 고양이연맹은 예쁜 고양이 대회를 비롯해 매년 700여 개의 고양이 관련 대회를 주최해왔다. 연맹 측은 오는 5월 말까지 러시아산 고양이의 수입을 금지하고 러시아에서 태어난 고양이의 혈통서 등록도 허용하지 않겠다며 우크라이나를 향한 러시아의 잔혹한 행위에 대해 더 이상 침묵할 수 없었다고 밝혔다. 이어 우크라이나 난민과 피란 중인 반려동물을 위한 기금도 마련할 것이라고 덧붙였다. 러시아 고양이의 국제 대회 출전 금지령에 대해 어떻게 생각하나요.'

In [34]:
df["split_article"][0]

['전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다',
 '이에 러시아 고양이도 국제 대회에 출전하지 못한다는 소식이 전해졌다',
 '지난 3일 영국 일간 데일리메일에 따르면 70여 년 역를 가 국제고양이연맹이 최근 성명을 통해 월드 캣 쇼에 러시아 소속 고양이 참가를 금지하겠다고 밝혔다',
 '고양이연맹은 예쁜 고양이 대회를 비롯해 매년 700여 개의 고양이 관련 대회를 주최해왔다',
 '연맹 측은 오는 5월 말까지 러시아산 고양이의 수입을 금지하고 러시아에서 태어난 고양이의 혈통서 등록도 허용하지 않겠다며 우크라이나를 향한 러시아의 잔혹한 행위에 대해 더 이상 침묵할 수 없었다고 밝혔다',
 '이어 우크라이나 난민과 피란 중인 반려동물을 위한 기금도 마련할 것이라고 덧붙였다',
 '러시아 고양이의 국제 대회 출전 금지령에 대해 어떻게 생각하나요.']

In [35]:
df["split_article"][0][0]

'전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다'

In [36]:
df.iloc[:, 4:]

,news_len,split_title,split_article
0,469,[세상 속으로],"[전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다, 이에 러시아 고..."
1,380,[뉴스탐험대],"[우크라이나 소년이 홀로 국경을 넘은 뭉클한 연이 전해졌다, 영국 매체 가디언과 메..."
2,446,[뉴스탐험대],[온라인 쇼핑몰에서 무알러지 무니켈 도금 등으로 홍보 판매되는 액세서리에서 기준치를...
3,451,[뉴스탐험대],[혈액형이 O형인 람이 코로나 바이러스 감염 시 망 위중증 위험이 다른 혈액형보다 ...
4,620,잡히지 않는 火魔<화마>… 문화재 등 대규모 피해,"[최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림 피해가 우려되고 있다, ..."
...,...,...,...
1522,1124,돼지 뇌에 컴퓨터 칩 연결… 생각만으로 기계 움직일 수 있을까?,[머스크 유튜브에 실험 돼지 거트루드 공개 뇌 질환 치료 도움 올해 안에 람 적용 ...
1523,721,탑승형 로봇 '프로스테시스' 조종사를 뽑습니다!,[람의 팔다리에 연결해 조종 로봇 활용한 경주 스포츠 만들 것 람이 탑승한 로봇끼리...
1524,1668,"모험 떠날 준비 됐니? 그럼, 책장을 펼쳐봐!","[띵똥, 선물이 도착했습니다, 판타지 소설 해리포터의 작가 조앤 롤링이 코로나19로..."
1525,756,"방탄소년단·블랙핑크, 세계 음원시장 '올킬'","[아이돌 그룹 방탄소년단과 블랙핑크가 세계 무대를 뒤흔들고 있다, 글로벌 시장을 겨..."


### nltk 활용

In [51]:
# Series to list type
#news_list = news_summary.news_article_clean.to_list()

In [37]:
from nltk.tokenize import sent_tokenize
import pprint
pp = pprint.PrettyPrinter(indent=0)

article_text = df.news_article[0]
# print('문장 토큰화1 :',sent_tokenize(text))
pp.pprint(sent_tokenize(article_text))

['전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다.',
'이에 러시아 고양이도 국제 대회에 출전하지 못한다는 소식이 전해졌다.',
'지난 3일 영국 일간 데일리메일에 따르면 70여 년 역를 가 국제고양이연맹이 최근 성명을 통해 월드 캣 쇼에 러시아 소속 고양이 참가를 '
'금지하겠다고 밝혔다.',
'고양이연맹은 예쁜 고양이 대회를 비롯해 매년 700여 개의 고양이 관련 대회를 주최해왔다.',
'연맹 측은 오는 5월 말까지 러시아산 고양이의 수입을 금지하고 러시아에서 태어난 고양이의 혈통서 등록도 허용하지 않겠다며 우크라이나를 향한 '
'러시아의 잔혹한 행위에 대해 더 이상 침묵할 수 없었다고 밝혔다.',
'이어 우크라이나 난민과 피란 중인 반려동물을 위한 기금도 마련할 것이라고 덧붙였다.',
'러시아 고양이의 국제 대회 출전 금지령에 대해 어떻게 생각하나요.']


---

## 4. 전처리 끝난 DataFrame 재생성
- 라벨링 판정 form에 맞춰 DataFrame 재생성

In [38]:
df = df[['news_title', 'news_article', 'split_article']]
df.head()

,news_title,news_article,split_article
0,러시아 고양이는 안 됩니다,전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다. 이에 러시아 고양...,"[전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다, 이에 러시아 고..."
1,홀로 1000 우크라 국경 탈출한 11세 소년,우크라이나 소년이 홀로 국경을 넘은 뭉클한 연이 전해졌다. 영국 매체 가디언과 메트...,"[우크라이나 소년이 홀로 국경을 넘은 뭉클한 연이 전해졌다, 영국 매체 가디언과 메..."
2,온라인 쇼핑몰 니켈 납투성이 장신구 주의,온라인 쇼핑몰에서 무알러지 무니켈 도금 등으로 홍보 판매되는 액세서리에서 기준치를 ...,[온라인 쇼핑몰에서 무알러지 무니켈 도금 등으로 홍보 판매되는 액세서리에서 기준치를...
3,코로나에 강한 혈액형 있다 O형 위험 최저,혈액형이 O형인 람이 코로나 바이러스 감염 시 망 위중증 위험이 다른 혈액형보다 낮...,[혈액형이 O형인 람이 코로나 바이러스 감염 시 망 위중증 위험이 다른 혈액형보다 ...
4,잡히지 않는 화마 문화재 등 대규모 피해,최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림 피해가 우려되고 있다. 지...,"[최근 발생한 동해안 지역 대규모 산불로 인해 막대한 산림 피해가 우려되고 있다, ..."


In [42]:
# 전체 dataframe에서 실행
total_data = []
for idxx in df.index:
    news_article = df.split_article[idxx]
    news_title = []
    news_index = []

    for i in range(len(news_article)):
        news_title.append(df.news_title[idxx])
        news_index.append(idxx + 1)
    for idx, title, article in zip(news_index, news_title, news_article): 
        total_data.append([idx,title,article]) 

In [44]:
news_summary = pd.DataFrame(columns = ['news_idx','news_title','news_article'],data=total_data)
news_summary

,news_idx,news_title,news_article
0,1,러시아 고양이는 안 됩니다,전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다
1,1,러시아 고양이는 안 됩니다,이에 러시아 고양이도 국제 대회에 출전하지 못한다는 소식이 전해졌다
2,1,러시아 고양이는 안 됩니다,지난 3일 영국 일간 데일리메일에 따르면 70여 년 역를 가 국제고양이연맹이 최근 ...
3,1,러시아 고양이는 안 됩니다,고양이연맹은 예쁜 고양이 대회를 비롯해 매년 700여 개의 고양이 관련 대회를 주최해왔다
4,1,러시아 고양이는 안 됩니다,연맹 측은 오는 5월 말까지 러시아산 고양이의 수입을 금지하고 러시아에서 태어난 고...
...,...,...,...
30829,1527,알파고가 이세돌 9단을 이긴 건 딥러닝 덕분이지,아마도 람이 이 방법으로 학습한다면 똑같은 과정을 거치는 데 1000년도 더 걸렸을걸
30830,1527,알파고가 이세돌 9단을 이긴 건 딥러닝 덕분이지,인터넷 검색 이트 구글은 이제 1000만 장의 에서 고양이 을 찾아낼 수 있대
30831,1527,알파고가 이세돌 9단을 이긴 건 딥러닝 덕분이지,우리에게는 너무 당연한 일이지만 예전에는 컴퓨터가 람 얼굴과 고양이 얼굴을 구별하는...
30832,1527,알파고가 이세돌 9단을 이긴 건 딥러닝 덕분이지,알파고처럼 학습한 끝에 고양이를 알아차릴 수 있게 됐지


---

## 5. 형태소 분석

### KoNLTK: Korean Natural Language ToolKit
- KoNLPy(“코엔엘파이”라고 읽습니다)는 한국어 정보처리를 위한 파이썬 패키지
- tokenizing

In [45]:
news_title = str(news_summary["news_title"][0])
news_article = str(news_summary["news_article"][0])
print('title:',news_title,'\narticle:',news_article)

title: 러시아 고양이는 안 됩니다 
article: 전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다


In [46]:
type(news_title)

str

### Mecab 형태소 분석기
- 일본어용 형태소 분석기를 한국어를 사용할 수 있도록 수정

In [47]:
from konlpy.tag import Mecab 
# from eunjeon import Mecab 
mecab = Mecab()

In [48]:
# 텍스트에서 형태소를 반환한다
print(mecab.morphs(news_title))

['러시아', '고양이', '는', '안', '됩니다']


In [49]:
# 텍스트에서 명사를 반환한다
print(mecab.nouns(news_title))

['러시아', '고양이']


In [50]:
# 텍스트에서 품사 정보를 부착하여 반환한다
print(mecab.pos(news_title))

[('러시아', 'NNP'), ('고양이', 'NNG'), ('는', 'JX'), ('안', 'MAG'), ('됩니다', 'VV+EF')]


---

### Kkma
- 서울대학교 IDS(intelligent Data Systems) 연구실에서 개발

In [51]:
from konlpy.tag import Kkma 
JVM_PATH = '/Library/Java/JavaVirtualMachines/zulu-15.jdk/Contents/Home/bin/java'
kkma = Kkma(jvmpath=JVM_PATH)

[0.017s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016cfb8000-0x000000016cfc4000).
[0.017s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [52]:
# 텍스트에서 형태소를 반환한다
print(kkma.morphs(news_title))

['러시아', '고양이', '는', '안', '되', 'ㅂ니다']


In [53]:
# 텍스트에서 명사를 반환한다
print(kkma.nouns(news_title))

['러시아', '고양이']


In [54]:
# 텍스트에서 품사 정보를 부착하여 반환한다
print(kkma.pos(news_title))

[('러시아', 'NNG'), ('고양이', 'NNG'), ('는', 'JX'), ('안', 'MAG'), ('되', 'VV'), ('ㅂ니다', 'EFN')]


---

### Okt (Twitter)
- 오픈 소스 한국어 분석기이고, 과거 트위터 형태소 분석기

In [55]:
# from konlpy.tag import Twitter
from konlpy.tag import Okt  
okt = Okt(jvmpath=JVM_PATH) 

In [56]:
# 텍스트에서 형태소를 반환한다
print(okt.morphs(news_title))

['러시아', '고양이', '는', '안', '됩니다']


In [57]:
# 텍스트에서 명사를 반환한다
print(okt.nouns(news_title))

['러시아', '고양이', '안']


In [58]:
# 텍스트에서 어절을 뽑아낸다
print(okt.phrases(news_title))

['러시아', '러시아 고양이', '고양이']


In [59]:
# 텍스트에서 품사 정보를 부착하여 반환한다
print(okt.pos(news_title))

[('러시아', 'Noun'), ('고양이', 'Noun'), ('는', 'Josa'), ('안', 'Noun'), ('됩니다', 'Verb')]


---

## 6. News Article 유사도 분석
- title과 article의 유사도 분석하기

In [60]:
idx_groups = news_summary.groupby('news_idx')
idx_groups.size()
#pandas.core.groupby.generic.DataFrameGroupBy
# type(idx_groups)
# idx_groups.sort_values(by='news_idx' ,ascending=True)

news_idx
1        7
2        8
3        9
4        7
5       12
        ..
1523    20
1524    13
1525    33
1526    11
1527    43
Length: 1527, dtype: int64

In [81]:
# 딕셔너리로 범주별로 저장
group_news_df = dict(list(idx_groups))
# pandas.core.frame.DataFrame
# type(group_news_df[1])
group_news_df[1][:2]

,news_idx,news_title,news_subtitle,news_article
0,1,러시아 고양이는 안 됩니다,"국제고양이연맹, '월드 캣 쇼'서 퇴출",전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다
1,1,러시아 고양이는 안 됩니다,"국제고양이연맹, '월드 캣 쇼'서 퇴출",이에 러시아 고양이도 국제 대회에 출전하지 못한다는 소식이 전해졌다


In [61]:
#==위와동일
news_summary[news_summary['news_idx']==1][:2]

,news_idx,news_title,news_article
0,1,러시아 고양이는 안 됩니다,전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다
1,1,러시아 고양이는 안 됩니다,이에 러시아 고양이도 국제 대회에 출전하지 못한다는 소식이 전해졌다


In [62]:
len(news_summary.index)

30834

In [63]:
%%time
total_news_title = []
total_news_article = []
for i in range(len(news_summary.index)) :
    news_title = news_summary[news_summary['news_idx']==i+1].news_title
    news_article = news_summary[news_summary['news_idx']==i+1].news_article
    total_news_title.append(news_title)
    total_news_article.append(news_article)

CPU times: user 15.9 s, sys: 172 ms, total: 16.1 s
Wall time: 8.89 s


In [64]:
print(total_news_title[0])

0    러시아 고양이는 안 됩니다
1    러시아 고양이는 안 됩니다
2    러시아 고양이는 안 됩니다
3    러시아 고양이는 안 됩니다
4    러시아 고양이는 안 됩니다
5    러시아 고양이는 안 됩니다
6    러시아 고양이는 안 됩니다
Name: news_title, dtype: object


In [65]:
print(total_news_article[0])

0                  전 세계가 우크라이나를 침공한 러시아에 각종 제재를 가하고 있다
1                이에 러시아 고양이도 국제 대회에 출전하지 못한다는 소식이 전해졌다
2    지난 3일 영국 일간 데일리메일에 따르면 70여 년 역를 가 국제고양이연맹이 최근 ...
3    고양이연맹은 예쁜 고양이 대회를 비롯해 매년 700여 개의 고양이 관련 대회를 주최해왔다
4    연맹 측은 오는 5월 말까지 러시아산 고양이의 수입을 금지하고 러시아에서 태어난 고...
5        이어 우크라이나 난민과 피란 중인 반려동물을 위한 기금도 마련할 것이라고 덧붙였다
6                 러시아 고양이의 국제 대회 출전 금지령에 대해 어떻게 생각하나요.
Name: news_article, dtype: object


---

### Mecab 형태소 분석

- news_article

In [66]:
%%time
# total_news_title
# total_news_article
total_group_news_article = []
for group_news_article in total_news_article :
    
    news_article_sentence = []
    for sentence in group_news_article:
        news_noun = ' '.join(mecab.nouns(sentence))
        news_article_sentence.append(news_noun)
    total_group_news_article.append(news_article_sentence)

CPU times: user 2.25 s, sys: 60.1 ms, total: 2.31 s
Wall time: 1.99 s


In [67]:
total_group_news_article[0]

['세계 우크라이나 침공 러시아 각종 제재',
 '이 러시아 고양이 국제 대회 출전 소식',
 '일 영국 일간 데일리 메일 년 역 국제 고양이 연맹 최근 성명 월드 캣 쇼 러시아 소속 고양이 참가 금지',
 '고양이 연맹 고양이 대회 개 고양이 관련 대회 주최',
 '연맹 측 월 말 러시아 고양이 수입 금지 러시아 고양이 혈통서 등록 허용 우크라이나 러시아 잔혹 행위 이상 침묵 수',
 '우크라 난민 피란 중 반려 동물 기금 마련 것',
 '러시아 고양이 국제 대회 출전 금지령 생각']

- news_title

In [68]:
%%time
# total_news_title
# total_news_article
total_group_news_title = []
for group_news_title in total_news_title :
    
    news_title_sentence = []
    for sentence in group_news_title:
        news_noun = ' '.join(mecab.nouns(sentence))
        news_title_sentence.append(news_noun)
    total_group_news_title.append(news_title_sentence)

CPU times: user 618 ms, sys: 8.76 ms, total: 627 ms
Wall time: 631 ms


In [69]:
total_group_news_title[0]

['러시아 고양이', '러시아 고양이', '러시아 고양이', '러시아 고양이', '러시아 고양이', '러시아 고양이', '러시아 고양이']

---